In [145]:
from tools.ground_removal import *
from tools.voxelization import *
from tools.pedestrians import *
from sklearn.cluster import DBSCAN
import numpy as np
from mayavi import mlab

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [219]:
pts_, intens = get_synchronized_frames(19,10)
#mask = (pts_[:,2] < 1) & (pts_[:,2] > -1)
#pts = pts_[mask]
pts = pts_

In [4]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()



In [220]:

mask = (pts_[:,2] > -1) & (pts_[:,2] < 1)
cropped = pts_[mask]

In [221]:
clustering = DBSCAN(eps=0.3, min_samples=7 * 10, ).fit(cropped[:, :3])

In [224]:
mlab.points3d(cropped[:,0], cropped[:,1], cropped[:,2],clustering.labels_, mode='point')
mlab.show()

In [159]:
a = np.array([[1,2,3],[2,2,2],[3,1,5]])
b = np.array([True, True, False])
b.shape

(3,)

In [291]:
pts_, intens = get_synchronized_frames(0,10)
mask = (pts_[:,2] > -1) & (pts_[:,2] < 1)
times = pts_[:,3].reshape(-1,1)
cropped_pts = pts_[:,:3][mask]
cropped_times = pts_[:,3][mask].reshape(-1,1)

dynamic_mask, clustering, dynamic_clusters, \
    differences, centroids_final \
    = find_pedestrians(pts_[:,:3], times, cropped_pts, cropped_times)

formed 61 clusters
skipping cluster 0 because of it's size w 17.39520876306575, l 3.316139942931465, h 1.9999017762832334
skipping cluster 1 at time 0 because of it's size w 8.773822545710932, l 4.459904671698093, h 1.511138498171964
skipping cluster 2 at time 0 because of it's size w 1.2360420224637627, l 4.689170360839952, h 1.4631000134304308
cluster 3, time 7 skipping because no points; cluster 3, time 8 skipping because no points; cluster 3, time 9 skipping because no points; 

cluster 4, time 7 skipping because no points; cluster 4, time 8 skipping because no points; cluster 4, time 9 skipping because no points; 

skipping cluster 5 because of it's size w 2.8124292468350127, l 3.9025969933977365, h 1.8787573205871393
skipping cluster 7 because of it's size w 8.45688004123883, l 2.5055140061902375, h 1.9967459086755133
skipping cluster 8 at time 0 because of it's size w 2.5805063248732214, l 0.47379827493114135, h 1.0698446923583855
skipping cluster 9 because of it's size w 2.8165

cluster 54 ground observed at times [5. 6.]
skipping cluster 54 because ground is only observed at times [5. 6.] at frequencies [7 3]
cluster 56 first time = 5.0
cluster 56 ground observed at times [6. 7.]
skipping cluster 56 because ground is only observed at times [6. 7.] at frequencies [ 2 15]
cluster 57 first time = 6.0
skipping cluster 57 because it is not observed in enough frames,first time of observation is 6.0
cluster 58 first time = 7.0
skipping cluster 58 because it is not observed in enough frames,first time of observation is 7.0
cluster 59 first time = 7.0
skipping cluster 59 because it is not observed in enough frames,first time of observation is 7.0
cluster 60 first time = 7.0
skipping cluster 60 because it is not observed in enough frames,first time of observation is 7.0


In [292]:
print(dynamic_clusters)


[33, 34, 36, 39, 53]


In [293]:
mlab.points3d(cropped_pts[:,0][dynamic_mask], cropped_pts[:,1][dynamic_mask],
              cropped_pts[:,2][dynamic_mask], cropped_times[dynamic_mask].reshape(-1,), mode='point')
mlab.points3d(cropped_pts[:,0][~dynamic_mask], cropped_pts[:,1][~dynamic_mask], 
              cropped_pts[:,2][~dynamic_mask], mode='point')
mlab.show()

In [209]:
single_mask.shape
cropped_times[single_mask][time_mask].shape

(270, 1)

In [249]:
cluster = 23  #dynamic_clusters[4]
print(f"average speed {np.linalg.norm(differences[cluster][:2])}")
single_mask = clustering.labels_ == cluster
time_mask = cropped_times[single_mask] == 19 # at time 0
time_mask = time_mask.reshape(-1,)
print(np.sum(time_mask))

mlab.points3d(cropped_pts[:,0][single_mask][time_mask], cropped_pts[:,1][single_mask][time_mask],
              cropped_pts[:,2][single_mask][time_mask], cropped_times[single_mask][time_mask].reshape(-1,),
              mode='point')

mlab.points3d(cropped_pts[:,0][~single_mask], cropped_pts[:,1][~single_mask], cropped_pts[:,2][~single_mask],
              mode='point')

#mlab.points3d(pts_[:,0], pts_[:,1], pts_[:,2], mode='point')
mlab.show()

average speed 0.190253726046953
494


In [250]:
bb_x_min =  cropped_pts[:,0][single_mask][time_mask].min()
bb_x_max =  cropped_pts[:,0][single_mask][time_mask].max()

bb_y_min =  cropped_pts[:,1][single_mask][time_mask].min()
bb_y_max =  cropped_pts[:,1][single_mask][time_mask].max()

bb_z_min =  cropped_pts[:,2][single_mask][time_mask].min()
bb_z_max =  cropped_pts[:,2][single_mask][time_mask].max()

In [235]:

# along x axis
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_min, bb_y_min], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_max, bb_y_max], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_min, bb_y_min], [bb_z_max, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_max], [bb_y_max, bb_y_max], [bb_z_max, bb_z_max], color=(1,0.7,0.3))

# along y axis
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_min, bb_y_max], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_min, bb_y_max], [bb_z_max, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_min, bb_y_max], [bb_z_min, bb_z_min], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_min, bb_y_max], [bb_z_max, bb_z_max], color=(1,0.7,0.3))

# along z axis
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_min, bb_y_min], [bb_z_min, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_min, bb_x_min], [bb_y_max, bb_y_max], [bb_z_min, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_min, bb_y_min], [bb_z_min, bb_z_max], color=(1,0.7,0.3))
mlab.plot3d([bb_x_max, bb_x_max], [bb_y_max, bb_y_max], [bb_z_min, bb_z_max], color=(1,0.7,0.3))


mlab.points3d(cropped_pts[:,0][single_mask][time_mask], cropped_pts[:,1][single_mask][time_mask],
              cropped_pts[:,2][single_mask][time_mask], cropped_times[single_mask][time_mask].reshape(-1,),
              mode='point')
mlab.show()

In [285]:
bb_mask = (pts_[:,0] < bb_x_max) & (pts_[:,0] > bb_x_min) \
    & (pts_[:,1] < bb_y_max) & (pts_[:,1] > bb_y_min)

floor = pts_[:,2][bb_mask].min()
floor = max(floor,bb_z_min - 0.4 )
bb_mask = bb_mask & (pts_[:,2] < (floor + 0.1))

In [286]:
print(bb_z_min)
print(floor)

-0.9930473361031125
-0.9930473361031125


In [287]:
mlab.points3d(pts_[:,0][bb_mask], pts_[:,1][bb_mask],
              pts_[:,2][bb_mask], pts_[:,3][bb_mask],
              mode='point')
mlab.points3d(pts_[:,0][~bb_mask], pts_[:,1][~bb_mask],
              pts_[:,2][~bb_mask],
              mode='point')
mlab.show()


In [552]:
np.sum(pts_[:,3][bb_mask] == 3)

0

In [108]:
v,f = np.unique(pts_[:,3][bb_mask], return_counts=True)
v[f>1]


array([2., 3., 6., 7., 8.])

In [109]:
pts_[:,3][bb_mask]

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 6., 6., 6.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8.])

In [569]:
for i,j in zip(v,f):
    print(i)

0.0
2.0
4.0
5.0
6.0
8.0
9.0


In [615]:
bb_mask = (pts_[:,0] < bb_x_max) & (pts_[:,0] > bb_x_min) \
    & (pts_[:,1] < bb_y_max) & (pts_[:,1] > bb_y_min) \
    & (pts_[:,2] < bb_z_max) & (pts_[:,2] > bb_z_min)
np.sum((pts_[:,3] == 5)[bb_mask])

4